# Поиск решения проблем 2 подхода

Основная проблема заключается в крайне малой метрике разработанной модели. Так как это моя вторая полностью самостоятельная нейросетевая модель, плюс первый раз делал трансформеры, то ничего удивительного.

Возможные причины:

1. В приницпе не подходящая архитектура. Возможно, трансформеры для этой задачи неприменимы или я не смогу их применить.
1. Плохо подготовленные данные. Для снижения их размерности пришлось немного данных убрать и немного обработать оставшиеся.
1. Неоптимальная архитектура. Без понятия как выбираются архитектуры сетей, сделал по аналогии с архитектурами из интернета.
1. Неправильная загрузка данных. Я загружаю пачками с одним значением целевого класса.
1. Ошибка на этапе обучения модели.

## Подходят ли трансформеры?

Самая простая проверка - заменить блок трансформера в сети на обычную сеть или сверточную сеть и посмотреть как изменится метрика. Это легко проверить.

In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from a2 import MyDataGenerator, get_n_fit_model

In [2]:
%%time
# сначала нужно подготовить один data generator на все тесты
data_generator = MyDataGenerator(model_batch_size=2 ** 4,
                     pca_var=0.9,
                     pca_sample_size=50000,
                     force_pca_fitting=True,
                     verbose=100)
print(data_generator.data_shape)

# и параметры для обучения моделей
fit_params = {'steps_per_epoch': 1000,
              'epochs': 20,
              'verbose': 1}

# каррируем функцию для оценки
test = lambda **kwargs: get_n_fit_model(data_generator=data_generator,
                                        model_fit_kwargs=fit_params,
                                        plot=True,
                                        **kwargs)

# результаты
result = dict()

load main dataframe (3.9 Gb)
load target dataframe (2 Mb)
load main dataframe in dict splitted by users (loaded)
(16, 660)
CPU times: total: 1min 24s
Wall time: 1min 58s


### Трансформер

In [3]:
%%time
result['transformer'] = test(model_kwargs={'multihead_count': 2,
                                           'heads': 10,
                                           'ff_count': 2,
                                           'ff_wide': 100,
                                           'ff_activation': 'relu',
                                           'last_ff_count': 2,
                                           'last_ff_wide': 100,
                                           'last_ff_activation': 'relu',
                                           'dropout': 0.1})

Epoch 1/20
 186/1000 [====>.........................] - ETA: 10:08 - loss: 0.7912 - binary_accuracy: 0.5044

KeyboardInterrupt: 

### ResNet

А почему бы и не попробовать?

In [4]:
def res_block(x, kernel_size, filters, strides, activation_layer_class):
    x = layers.Conv2D(kernel_size=1,
                      filters=filters)(x)
    
    y = layers.Conv2D(kernel_size=kernel_size,
                      strides=strides,
                      filters=filters,
                      padding="same")(x)
    y = activation_layer_class()(y)
    y = layers.Conv2D(kernel_size=kernel_size,
                      strides=1,
                      filters=filters,
                      padding="same")(y)
    y = activation_layer_class()(y)
    if strides > 1:
        x = layers.Conv2D(kernel_size=2,
                          strides=strides,
                          filters=filters,
                          padding="same")(x)
    x = layers.Add()([x, y])
    x = activation_layer_class()(x)
    return x

def get_resnet_model(data_shape, kernel_size=3,
                     filters=4, activation_res='relu',
                     res_block_count=2, res_block_repeat=2,
                     strides=1,
                     dense_count=2, dense_wide=10,
                     dropout=0.1):
    data_shape = list(data_shape) + [1]
    activation_res_layer_class = {'relu': layers.ReLU,
                                  'elu': layers.ELU,}[activation_res]
    
    inputs = keras.Input(shape=data_shape)
    x = inputs
    
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    for i in range(res_block_count):
        for j in range(res_block_repeat):
            x = res_block(x,
                          kernel_size=kernel_size,
                          strides=strides if j == 0 and i != 0 else 1,
                          filters=filters * (2 ** i),
                          activation_layer_class=activation_res_layer_class)
    x = layers.Flatten()(x)
    
    for i in range(dense_count):
        x = layers.Dense(dense_wide // (i + 1), activation='elu')(x)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)

    model.compile(loss=keras.losses.BinaryCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=[keras.metrics.BinaryAccuracy()])
    return model

In [5]:
result['resnet'] = test(get_model_fun=get_resnet_model,
                             model_kwargs={'kernel_size': 3,
                                           'filters': 4,
                                           'activation_res': 'relu',
                                           'res_block_count': 2,
                                           'res_block_repeat': 2,
                                           'strides': 2,
                                           'dense_count': 2,
                                           'dense_wide': 10,
                                           'dropout': 0.1})

Epoch 1/20



KeyboardInterrupt



In [6]:
def get_dense_model(data_shape, layers_count=2,
                    layers_wide=3000,
                    activation='elu',
                    dropout=0.1):
    data_shape = list(data_shape) + [1]
    
    inputs = keras.Input(shape=data_shape)
    x = inputs
    x = layers.Flatten()(x)
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    
    for i in range(layers_count):
        x = layers.Dense(layers_wide - (layers_wide // layers_count) * i, activation=activation)(x)
        x = layers.Dropout(dropout)(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    model = keras.Model(inputs, outputs)

    model.compile(loss=keras.losses.BinaryCrossentropy(),
                  optimizer=keras.optimizers.Adam(learning_rate=1e-4),
                  metrics=[keras.metrics.BinaryAccuracy()])
    return model

In [ ]:
# model = get_dense_model(data_generator.data_shape)
# model.summary()

In [7]:
result['resnet'] = test(get_model_fun=get_dense_model,
                             model_kwargs={'layers_count': 3,
                                           'layers_wide': 2000,
                                           'activation': 'elu',
                                           'dropout': 0.1})

Epoch 1/20
1000/1000 [==============================] - 199s 198ms/step - loss: 0.9866 - binary_accuracy: 0.5027
Epoch 2/20
1000/1000 [==============================] - 197s 197ms/step - loss: 0.8199 - binary_accuracy: 0.4926
Epoch 3/20
 510/1000 [==============>...............] - ETA: 1:37 - loss: 0.7917 - binary_accuracy: 0.4640

KeyboardInterrupt: 